Guidance on using this dataset: https://www.kaggle.com/code/acelevin/identifying-playing-cards

Download dataset from: https://www.kaggle.com/datasets/gunhcolab/object-detection-dataset-standard-52card-deck/data

In [1]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 10

### My attempt at a ResNet Network Model

In [18]:
class ResidualBlock(tf.keras.layers.Layer):
    ''' Res Block for an attempt at a ResNet architecture clone'''
    def __init__(self, filters, downsample=False):
        super(ResidualBlock, self).__init__()
        self.filters = filters
        self.downsample = downsample

        # downsample will halve the image size
        strides = 2 if downsample else 1

        self.conv1 = tf.keras.layers.Conv2D(filters, 3, strides=strides, padding='same', use_bias=False)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()

        self.conv2 = tf.keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False)
        self.bn2 = tf.keras.layers.BatchNormalization()

        # Optional downsampling for the shortcut path
        if downsample:
            self.downsample_conv = tf.keras.layers.Conv2D(filters, 1, strides=2, padding='same', use_bias=False)
            self.downsample_bn = tf.keras.layers.BatchNormalization()
        else:
            self.downsample_conv = None
            
    def call(self, inputs, training=False):
        shortcut = inputs

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        if self.downsample_conv:
            shortcut = self.downsample_conv(shortcut)
            shortcut = self.downsample_bn(shortcut, training=training)

        x = tf.keras.layers.add([x, shortcut])
        return self.relu(x)
        

class MockResNet(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [
            tf.keras.Input(shape=(300, 300, 3)),
            
            tf.keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),

            ResidualBlock(64),
            ResidualBlock(64),

            ResidualBlock(128, downsample=True),
            ResidualBlock(128),

            ResidualBlock(256, downsample=True),
            ResidualBlock(256),

            ResidualBlock(512, downsample=True),
            ResidualBlock(512),

            tf.keras.layers.GlobalAveragePooling2D(),
            
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ]

### Traditional CNN Model

In [17]:
class CardPredictor(tf.keras.Model):
    def __init__(self):
        super(CardPredictor, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                data_augmentation,
                             
                tf.keras.layers.Conv2D(32, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(64, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(128, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(256, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(512, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Flatten(),                                   # Flatten to vector
                
                tf.keras.layers.Dense(256, activation='relu'),               # Fully connected layer
                tf.keras.layers.Dropout(0.5),                                # Prevent overfitting
                tf.keras.layers.Dense(52, activation='softmax')
                ]
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor")
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

    @staticmethod
    def loss_fn(labels, predictions): 
           """ Loss function for the model. """
           return tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)

### Dataset Loading

In [13]:
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)
    
new_data = {}
for key, inner_dict in data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['class_label']
    new_data[img_path] = value

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((new_data.keys(), new_data.values()))

def load_train_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(load_train_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

### Model Loading and Training

In [15]:
#model = CardPredictor()
model = MockResNet()
#model = tf.keras.models.load_model("model_weights_89.keras")
model.compile(optimizer=model.optimizer, loss=model.loss_fn, metrics=['accuracy'])

In [16]:
model.fit(train_dataset, 
          validation_data=val_dataset, 
          epochs=NUM_EPOCHS, 
          verbose=1)

Epoch 1/10


2025-04-24 17:22:59.131695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


175/175 ━━━━━━━━━━━━━━━━━━━━ 61s 320ms/step - accuracy: 0.0841 - loss: 3.4123 - val_accuracy: 0.0549 - val_loss: 4.0381
Epoch 2/10
 68/175 ━━━━━━━━━━━━━━━━━━━━ 26s 245ms/step - accuracy: 0.0648 - loss: 3.5317

KeyboardInterrupt: 

In [49]:
model.save('model_weights_89.keras')  # can also use .h5 extension if preferred